# 连接到drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 安装conda

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda --version

# 安装软件

sratools

In [ ]:
%cd /content/software/sratools
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.2.1/sratoolkit.3.2.1-ubuntu64.tar.gz
!tar -zxvf sratoolkit.3.2.1-ubuntu64.tar.gz

fastp

In [ ]:
%cd /content/software/fastp
!wget http://opengene.org/fastp/fastp
!chmod a+x ./fastp

hisat2

In [ ]:
%cd /content/software/hisat2
!git clone https://github.com/DaehwanKimLab/hisat2.git
%cd hisat2
!make

featurecount

https://github.com/ShiLab-Bioinformatics/subread/releases
留作习题

# 测试数据下载

In [ ]:
%cd /content/drive/MyDrive/data/rnaseq_example
!/content/software/sratools/sratoolkit.3.2.1-ubuntu64/bin/prefetch SRR957680
!/content/software/sratools/sratoolkit.3.2.1-ubuntu64/bin/fastq-dump --split-files SRR957680

In [ ]:
!cat /content/drive/MyDrive/data/rnaseq_example/SRR957680_1.fastq | head -1000000 > test.fastq

In [ ]:
!cp /content/drive/MyDrive/data/rnaseq_example/test.fastq /content/test

# 参考数据下载

https://storage.cloud.google.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta

https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_47/gencode.v47.basic.annotation.gtf.gz

# 第一步 质控加过滤

In [ ]:
%cd /content/test
!/content/software/fastp/fastp -i test.fastq -o test_qc.fastq

# 第二步 比对

In [ ]:
# 比对之前需要用软件hisat2对人类基因组构建索引 只运行一次
%cd /content/drive/MyDrive/refdata/GATK_hg38
!/content/software/hisat2/hisat2/hisat2-build hg38.fa hg38_index

In [18]:
%cd /content/test
!/content/software/hisat2/hisat2/hisat2 \
-x /content/drive/MyDrive/refdata/GATK_hg38/hg38_index \
-U test_qc.fastq -S test.sam

/content/test
248903 reads; of these:
  248903 (100.00%) were unpaired; of these:
    7991 (3.21%) aligned 0 times
    206825 (83.09%) aligned exactly 1 time
    34087 (13.69%) aligned >1 times
96.79% overall alignment rate


In [20]:
!cat /content/test/test.sam | tail -10

SRR957680.249994	16	chr19	10677297	60	14M1302N36M	*	0	0	AGTATTAGCTGGAGAAACGCTATCAGTCAACGACCCCCCGGACGTTCTGG	DDDDDDDDDCDDDDDDDDDDDDDDDDBDDDD@9DJJJHHHHHFFFFFCCC	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:50	YT:Z:UU	XS:A:+	NH:i:1
SRR957680.249995	16	chr10	56357989	60	50M	*	0	0	TATTGGCTTCTGAGTATCTGTATTAATAGTTGTTCCTGCCAGCATATGAA	D9DDIEECD9BDDA@E9BED<@AFIEF>FFEF;;EE3DD?AC4DD?D???	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:50	YT:Z:UU	NH:i:1
SRR957680.249996	0	chr11	70355789	60	23M349N27M	*	0	0	TGGCAAAGAAGCATTAGGACAAGCTGGTGTTTCCGAGACGGATAACTCAT	1:==AD8?FBD;?@?CBGHFAG>3CFGEAGFGI9?C@)8DGF:@<=<=CC	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:50	YT:Z:UU	XS:A:+	NH:i:1
SRR957680.249997	0	chr18	6462542	1	50M	*	0	0	GGCGTCCAGTGAGGATGATCAGAATGGTCCCGGGGGGAATGCTGGCTCGC	@@@DDDD?D2ADFEEII<EHHGIIII9F9BEFGIII&058@8>>ABC###	AS:i:-2	ZS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:36T13	YT:Z:UU	NH:i:4
SRR957680.249997	272	chr7	137722406	1	50M	*	0	0	GCGAGCCAGCATTCCCCCCGGGACCATTCTGATCATCCTCACTGGACGCC	###C

# 生成count

In [ ]:
%cd /content/test
!/content/software/subread/subread-2.0.2-Linux-x86_64/bin/featureCounts \
  -a /content/drive/MyDrive/refdata/genecode/gencode.v47.basic.annotation.gtf \
  -o gene_counts.txt \
  test.sam

In [28]:
!cat /content/drive/MyDrive/refdata/genecode/gencode.v47.basic.annotation.gtf | head -10

##description: evidence-based annotation of the human genome (GRCh38), version 47 (Ensembl 113)
##provider: GENCODE
##contact: gencode-help@ebi.ac.uk
##format: gtf
##date: 2024-07-19
chr1	HAVANA	gene	11121	24894	.	+	.	gene_id "ENSG00000290825.2"; gene_type "lncRNA"; gene_name "DDX11L16"; level 2; tag "overlaps_pseudogene";
chr1	HAVANA	transcript	11426	14409	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST00000832828.1"; gene_type "lncRNA"; gene_name "DDX11L16"; transcript_type "lncRNA"; transcript_name "DDX11L16-264"; level 2; tag "basic"; tag "TAGENE";
chr1	HAVANA	exon	11426	11671	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST00000832828.1"; gene_type "lncRNA"; gene_name "DDX11L16"; transcript_type "lncRNA"; transcript_name "DDX11L16-264"; exon_number 1; exon_id "ENSE00004248702.1"; level 2; tag "basic"; tag "TAGENE";
chr1	HAVANA	exon	12010	12227	.	+	.	gene_id "ENSG00000290825.2"; transcript_id "ENST00000832828.1"; gene_type "lncRNA"; gene_name "DDX11L16"; transcript_typ